<a href="https://colab.research.google.com/github/zasuo-michikusa/Google_Colab_StableDiffusion/blob/main/StableDiffusion_AnimateDiff_custom_ipynb_20240103.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

★事前準備★

ご自分の「Googleドライブ」の「MyDrive」に 以下のフォルダ構成と

任意でGoogleColab上の「Stable-Diffusion_WEB-UI（Forge）」で使用したいデータを格納してください。

MyDrive

└StableDiffusion

 　└checkpoints　※使用したいモデルデータを格納

 　└controlnet　※ツール用モデルを格納

 　└embeddings　※EasyNegativeなどを格納

 　└lora　※使用したいLoRAデータを格納

 　└VAE　※使用したいVAEデータを格納

 　└outputs　※生成データのコピー用受け皿フォルダ（フォルダのみでOK）

★このノートブックの使い方★

～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～

①まずメニュー項目の「ランタイム」から
 → ランタイムの変更（いずれかのGPUタイプを選択）

②【01】の実行ボタン（▶）を押す、

（▶）の左に✅が付いたら次の（▶）を順番に押していく

※または「ランタイム」から「すべてのセルを実行」を実行する

③【06】の処理が進み、 https://************.gradio.live
と表示されたら、そのURLをクリック

④WEB-UIを停止するときは【05】の（▶）を押して処理をクルクルを停止

⑤再度WEB-UIを使用する際は、【05】の（▶）を押し③と同様

⑥生成したデータを保存する場合は【06】の（▶）を実行

～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～


以下、「StableDiffusion_AnimateDiff環境」の起動手順です。

👇【01】Googleドライブのデータをこのノートブックへマウント（接続）

※Googleドライブにある、checkpoint、LoRA、VAE、Embedding、ControlNet用のツールモデル等を参照できるようにする

↓の（▶）を押し、出現するポップアップウィンドウから

　 Googleドライブに接続 → 「アカウントの選択」 → 「アクセスを許可」

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


👇【02】WEB-UIと各種_拡張機能のクローン（起動前にあらかじめ導入）※お好みで増減可能

In [2]:
# Stable Diffusion Web-UI Forge のクローン（インストール）
!git clone https://github.com/lllyasviel/stable-diffusion-webui-forge.git
%cd /content/stable-diffusion-webui-forge

# Styles-Editorのクローン
%cd /content/stable-diffusion-webui-forge/extensions
!git clone https://github.com/chrisgoringe/Styles-Editor.git
%cd /content/stable-diffusion-webui-forge

# adetailerのクローン
%cd /content/stable-diffusion-webui-forge/extensions
!git clone https://github.com/Bing-su/adetailer.git
%cd /content/stable-diffusion-webui-forge

# aspect-ratio-helperのクローン
%cd /content/stable-diffusion-webui-forge/extensions
!git clone https://github.com/thomasasfk/sd-webui-aspect-ratio-helper.git
%cd /content/stable-diffusion-webui-forge

# prompt-all-in-oneのクローン
%cd /content/stable-diffusion-webui-forge/extensions
!git clone https://github.com/Physton/sd-webui-prompt-all-in-one.git
%cd /content/stable-diffusion-webui-forge

# ultimate-upscale-for-automatic1111のクローン
%cd /content/stable-diffusion-webui-forge/extensions
!git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111.git
%cd /content/stable-diffusion-webui-forge

# SegmentAnythingのクローン
%cd /content/stable-diffusion-webui-forge/extensions
!git clone https://github.com/continue-revolution/sd-webui-segment-anything.git
%cd /content/stable-diffusion-webui-forge

# AnimateDiff（Forge版）のクローン
%cd /content/stable-diffusion-webui-forge/extensions
!git clone https://github.com/continue-revolution/sd-forge-animatediff.git /content/stable-diffusion-webui-forge/extensions/sd-webui-animatediff
%cd /content/stable-diffusion-webui-forge

# Deforumのクローン
%cd /content/stable-diffusion-webui-forge/extensions
!git clone https://github.com/deforum-art/sd-webui-deforum.git
%cd /content/stable-diffusion-webui-forge


Cloning into 'stable-diffusion-webui-forge'...
remote: Enumerating objects: 37946, done.
remote: Counting objects: 100% (2787/2787), done.
remote: Compressing objects: 100% (902/902), done.
remote: Total 37946 (delta 2062), reused 2380 (delta 1839), pack-reused 35159
Receiving objects: 100% (37946/37946), 43.96 MiB | 27.60 MiB/s, done.
Resolving deltas: 100% (25963/25963), done.
/content/stable-diffusion-webui-forge
/content/stable-diffusion-webui-forge/extensions
Cloning into 'Styles-Editor'...
remote: Enumerating objects: 739, done.
remote: Counting objects: 100% (223/223), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 739 (delta 112), reused 104 (delta 87), pack-reused 516
Receiving objects: 100% (739/739), 143.52 KiB | 2.87 MiB/s, done.
Resolving deltas: 100% (389/389), done.
/content/stable-diffusion-webui-forge
/content/stable-diffusion-webui-forge/extensions
Cloning into 'adetailer'...
remote: Enumerating objects: 2567, done.
remote: Counting objects: 10

👇【03】MP4形式の出力用に「ffpeg」をインストール

In [ ]:
%cd /content/stable-diffusion-webui-forge/
!venv\Scripts\activate
!pip install imageio[ffmpeg]

👇【04】GoogleドライブへHugingFace、CIVITAIから各種必要なモデルデータをDL

「AnimateDiff」用の「MotionModel」

「SegmentAnything」用の「sam」「GroundingDINO」

In [ ]:
%cd /content/stable-diffusion-webui-forge/

!wget https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd14_v1.safetensors --directory-prefix=/content/stable-diffusion-webui-forge/extensions/sd-webui-animatediff/model

!wget https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd15_v1.safetensors --directory-prefix=/content/stable-diffusion-webui-forge/extensions/sd-webui-animatediff/model

!wget https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd15_v2.safetensors --directory-prefix=/content/stable-diffusion-webui-forge/extensions/sd-webui-animatediff/model

!wget https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd15_v3.safetensors --directory-prefix=/content/stable-diffusion-webui-forge/extensions/sd-webui-animatediff/model

!wget https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sdxl_hs.safetensors?download=true --directory-prefix=/content/stable-diffusion-webui-forge/extensions/sd-webui-animatediff/model

!wget https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sdxl_v10_beta.safetensors?download=true --directory-prefix=/content/stable-diffusion-webui-forge/extensions/sd-webui-animatediff/model

!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth --directory-prefix=/content/stable-diffusion-webui-forge/extensions/sd-webui-segment-anything/models/sam

!wget https://huggingface.co/ShilongLiu/GroundingDINO/resolve/main/GroundingDINO_SwinB.cfg.py?download=true --directory-prefix=/content/stable-diffusion-webui-forge/extensions/sd-webui-segment-anything/models/grounding-dino

!wget https://huggingface.co/ShilongLiu/GroundingDINO/resolve/main/GroundingDINO_SwinT_OGC.cfg.py?download=true --directory-prefix=/content/stable-diffusion-webui-forge/extensions/sd-webui-segment-anything/models/grounding-dino

!wget https://huggingface.co/ShilongLiu/GroundingDINO/resolve/main/groundingdino_swint_ogc.pth?download=true --directory-prefix=/content/stable-diffusion-webui-forge/extensions/sd-webui-segment-anything/models/grounding-dino

!wget https://huggingface.co/ShilongLiu/GroundingDINO/resolve/main/groundingdino_swinb_cogcoor.pth?download=true --directory-prefix=/content/stable-diffusion-webui-forge/extensions/sd-webui-segment-anything/models/grounding-dino

#下記必要な場合は場合は先頭の「!wget」の前の「#」を削除して有効化してください

#StableVideoDiffsion用のモデル
#!wget https://civitai.com/api/download/models/329995 --directory-prefix=/content/stable-diffusion-webui-forge/models/svd

#Zero123用のモデル
#!wget https://huggingface.co/stabilityai/stable-zero123/resolve/main/stable_zero123.ckpt?download=true --directory-prefix=/content/stable-diffusion-webui-forge/models/z123




👇【05】WEB-UIの起動（各種大容量データはGoogleドライブを参照する設定）

 ※生成した画像は自動的に/content/drive/MyDrive/StableDiffusion/outputs へ保存されます

 ※起動オプション追記もお好みで可能です

★ ↓の（▶）で WEB-UIの「起動」「停止」「再起動」を操作します。



In [ ]:
# スクリプトの内容を全体として実行
!git clone https://github.com/lllyasviel/stable-diffusion-webui-forge.git
%cd /content/stable-diffusion-webui-forge

# 各種拡張機能のクローン
%cd /content/stable-diffusion-webui-forge/extensions
!git clone https://github.com/chrisgoringe/Styles-Editor.git
!git clone https://github.com/Bing-su/adetailer.git
!git clone https://github.com/thomasasfk/sd-webui-aspect-ratio-helper.git
!git clone https://github.com/Physton/sd-webui-prompt-all-in-one.git
!git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111.git
!git clone https://github.com/continue-revolution/sd-webui-segment-anything.git
!git clone https://github.com/continue-revolution/sd-forge-animatediff.git /content/stable-diffusion-webui-forge/extensions/sd-webui-animatediff
!git clone https://github.com/deforum-art/sd-webui-deforum.git
%cd /content/stable-diffusion-webui-forge

# 出力ディレクトリを設定するスクリプトを作成
script_content = """
import os
from modules import shared

# Google Driveの出力ディレクトリを設定
shared.opts.data['outdir_txt2img_samples'] = '/content/drive/MyDrive/StableDiffusion/outputs/txt2img-images'
shared.opts.data['outdir_txt2img_grids'] = '/content/drive/MyDrive/StableDiffusion/outputs/txt2img-grids'
shared.opts.data['outdir_img2img_samples'] = '/content/drive/MyDrive/StableDiffusion/outputs/img2img-images'
shared.opts.data['outdir_img2img_grids'] = '/content/drive/MyDrive/StableDiffusion/outputs/img2img-grids'
shared.opts.data['outdir_extras_images'] = '/content/drive/MyDrive/StableDiffusion/outputs/extras-images'

print("出力ディレクトリの設定が完了しました。")
"""

# スクリプトを保存
script_path = "/content/stable-diffusion-webui-forge/scripts/set_output_dir.py"
with open(script_path, "w") as f:
    f.write(script_content)

# 設定スクリプトの実行とWEB-UIの起動
!python /content/stable-diffusion-webui-forge/scripts/set_output_dir.py
!python launch.py --share --no-half-vae --disable-nan-check \
  --ckpt-dir "/content/drive/MyDrive/StableDiffusion/checkpoints" \
  --embeddings-dir "/content/drive/MyDrive/StableDiffusion/embeddings" \
  --lora-dir "/content/drive/MyDrive/StableDiffusion/lora" \
  --vae-dir "/content/drive/MyDrive/StableDiffusion/vae" \
  --controlnet-dir "/content/drive/MyDrive/StableDiffusion/controlnet"

